In [1]:
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

In [2]:
# File to Load
assets_data_to_load = "assets.csv"
demo_data_to_load = "demographics.csv"
plays_data_to_load = "plays.csv"
psychographics_data_to_load = "psychographics.csv"
users_data_to_load = "users.csv"

# Read data into df's
assets_df = pd.read_csv(assets_data_to_load)
demo_data_df = pd.read_csv(demo_data_to_load)
psychographics_df = pd.read_csv(psychographics_data_to_load)
plays_df = pd.read_csv(plays_data_to_load)
users_df = pd.read_csv(users_data_to_load)

In [3]:
#Look over assets data to see what needs to be cleaned:
assets_df.head(20)

,show_type,genre,running_minutes,source_language,asset_id,season_id,series_id,studio_id
0,Movies,Sci-Fi,146,English,1,NaN,NaN,325.0
1,TV,Documentary and Biography,43,English,2,4.0,5.0,7.0
2,TV,Reality,22,English,3,15.0,22.0,442.0
3,TV,Reality,22,English,4,15.0,22.0,442.0
4,TV,Reality,22,English,5,15.0,22.0,442.0
5,TV,Comedy,23,English,6,12.0,20.0,397.0
6,TV,Comedy,23,English,7,13.0,20.0,397.0
7,TV,Comedy,23,English,8,13.0,20.0,397.0
8,TV,Kids,12,English,9,50.0,6.0,47.0
9,TV,Comedy,19,English,10,35.0,41.0,442.0


In [4]:
assets_df.dtypes

show_type           object
genre               object
running_minutes      int64
source_language     object
asset_id             int64
season_id          float64
series_id          float64
studio_id          float64
dtype: object

In [5]:
#Get count of Movies or TV
assets_df.groupby(["show_type"]).count()["asset_id"]

show_type
Movies     3293
TV        13978
Name: asset_id, dtype: int64

No data-cleaning needed for assets.csv.

In [6]:
#Look over plays data to see what needs to be cleaned:
plays_df.head(20)

,user_id,platform,asset_id,minutes_viewed
0,764504178919,android,13758,28
1,412316860611,android,13825,28
2,1503238553613,iOS,93,105
3,489626272003,android,6226,7
4,68719476744,android,3762,1
5,257698037999,android,4673,44
6,1236950581438,android,10526,1
7,1082331758653,android,14441,0
8,1219770712222,android,4808,28
9,755914244185,android,15019,11


In [7]:
#Get count of platform.
plays_df.groupby(["platform"]).count()["user_id"]

platform
Netrange             48
Panasonic           184
Roku                744
Samsung Orsay       769
Samsung Tizen      4029
Vewd                734
android          400266
android-tv        59678
googlecast          409
iOS               33071
web               27591
web-embed         10812
web-pwa             973
webOS              2849
Name: user_id, dtype: int64

In [8]:
plays_df.dtypes

user_id            int64
platform          object
asset_id           int64
minutes_viewed     int64
dtype: object

In [9]:
#Organize the plays data by ascending user_id order
sorted_plays_df = plays_df.sort_values(by=['user_id'])
sorted_plays_df

,user_id,platform,asset_id,minutes_viewed
123656,1,android,5727,0
316045,1,android,12060,4
249083,2,android,9233,0
307298,2,android,6637,0
101975,2,android,13778,53
...,...,...,...,...
194849,1709396984329,android,16915,21
503988,1709396984330,android,15097,1
269615,1709396984331,android,14936,108
527580,1709396984331,android,17106,84


In [10]:
#Get sum of minutes_viewed per user
plays_sum_df = plays_df.groupby("user_id").sum()["minutes_viewed"]
plays_sum_df

user_id
1                  4
2                233
3                 24
4                147
5                257
                ... 
1709396984327     14
1709396984328    103
1709396984329     34
1709396984330      1
1709396984331    207
Name: minutes_viewed, Length: 89602, dtype: int64

In [11]:
#Export the plays_sum_df to csv
plays_sum_df.to_csv('plays_sum.csv')

In [12]:
# writing to Excel
datatoexcel = pd.ExcelWriter('plays_sum.xlsx')
  
# write DataFrame to excel
plays_sum_df.to_excel(datatoexcel)
  
# save the excel
datatoexcel.save()

In [13]:
#Look over users data to see what needs to be cleaned:
users_df.head(20)

,user_id,country_code
0,781684047872,ID
1,781684047873,MY
2,781684047874,ID
3,781684047875,ID
4,781684047876,ID
5,781684047877,ID
6,781684047878,ID
7,781684047879,ID
8,781684047880,MY
9,781684047881,ID


In [14]:
#Get count of users by country code
users_df.groupby(["country_code"]).count()["user_id"]

country_code
BD     3117
BH       10
BN       47
EG      117
ET        1
GB        1
GH       78
ID    55240
IN        2
IQ      111
JO       26
JP        2
KE      123
KH      535
KR        1
KW       22
LA        3
LB       21
LK      306
MA      151
MM      581
MU       13
MV       12
MY    16798
NG       90
NP     8332
PH    19752
PK     1438
QA        1
SA      208
SD       32
SG        5
TH     2266
TN        1
TZ       33
UG       23
US        2
VN      942
ZM       14
ZW      182
Name: user_id, dtype: int64

Nothing of signficance that needs to be cleaned from users.csv.

In [15]:
#Look over psychographics data to see what needs to be cleaned:
psychographics_df.head(10)

,user_id,platform,level_1,level_2,level_3,confidence_score
0,352187318551,android,Psychographics,Social Media Fans,"\""\""",1.00
1,51539607784,android,Psychographics,Social Media Fans,"\""\""",1.00
2,1322849927469,android,Psychographics,Social Media Fans,"\""\""",1.00
3,137438953840,android,Psychographics,Movies Lovers,Romance Movies Fans,0.56
4,927712935974,android,Psychographics,Movies Lovers,Comedy Movies Fans,0.80
5,498216206346,android-tv,Psychographics,Movies Lovers,Horror Movies Fans,0.97
6,137438953842,web,Psychographics,Movies Lovers,Action and Adventure Movies Fans,0.37
7,1331439862003,android,Psychographics,Movies Lovers,Comedy Movies Fans,0.11
8,1374389535013,android,Psychographics,Movies Lovers,Family Movies Fans,0.27
9,816043786471,android,Psychographics,Movies Lovers,Family Movies Fans,0.42


In [16]:
#Sort psychographics data by user_id, in ascending order, and confidence_score than in descending order. This puts the
#the demographic characteristic that we are most confident with (higher confident score) at the top.
sorted_user_pg_df = psychographics_df.sort_values(['user_id', 'confidence_score'], ascending=[True, False])
sorted_user_pg_df

,user_id,platform,level_1,level_2,level_3,confidence_score
459980,0,iOS,Psychographics,iflix Viewing Behaviour,casual,1.00
232013,0,iOS,Psychographics,Travellers,Local Commuters,0.29
300772,1,android,Psychographics,iflix Viewing Behaviour,casual,1.00
455792,1,android,Psychographics,Movies Lovers,Documentary and Biography Movies Fans,0.61
370778,1,android,Psychographics,Movies Lovers,English Movies Fans,0.24
...,...,...,...,...,...,...
427460,1709396984331,android,Psychographics,TV Lovers,Drama TV Fans,0.61
145889,1709396984331,android,Psychographics,TV Lovers,Korean TV Fans,0.60
438133,1709396984331,android,Psychographics,Movies Lovers,Indonesian Movies Fans,0.58
314367,1709396984332,android,Psychographics,Mobile Enthusiasts,High Data Users,0.03


In [17]:
#Drop all duplicates (if applicable) and keeping only the most confidently-assessed characteristic (based on confidence_score).
preferences_df = sorted_user_pg_df.drop_duplicates(subset=['user_id'], keep='first', inplace=False)

In [18]:
#Get count of characteristics in level_2
sorted_user_pg_df.groupby(["level_2"]).count()["user_id"]

level_2
Mobile Enthusiasts              62860
Movies Lovers                  206494
Music Lovers                     2159
News Junkies & Avid Readers      1616
Social Media Fans                6247
Sports Fans                      3972
TV Lovers                       90932
Travellers                      55821
iflix Viewing Behaviour         44988
Name: user_id, dtype: int64

In [19]:
#Get count of characteristics in level_3
sorted_user_pg_df.groupby(["level_3"]).count()["user_id"]

level_3
Action and Adventure Movies Fans    12976
Action and Adventure TV Fans         1065
Adult Romance Movies Fans             345
Afrikaans Movies Fans                   2
Afrikaans TV Fans                       8
                                    ...  
addict                               3217
casual                              29555
eSports Movies Fans                    36
eSports TV Fans                        41
player                              12216
Name: user_id, Length: 121, dtype: int64

In [20]:
#Look over demographics data to see what needs to be cleaned:
demo_data_df.head(10)

,user_id,platform,level_1,level_2,level_3,confidence_score
0,171798691973,android,Demographics,Income,Medium,1.0
1,326417514592,android,Demographics,Income,Medium,1.0
2,17179869245,android,Demographics,Income,Medium,1.0
3,996432412842,android,Demographics,Income,Low,1.0
4,1606317768894,android,Demographics,Income,Low,1.0
5,1279900254421,iOS,Demographics,Income,High,1.0
6,1279900254249,android,Demographics,Income,High,1.0
7,8589934678,android,Demographics,Income,Low,1.0
8,1563368095824,android,Demographics,Income,Low,1.0
9,901943132374,android,Demographics,Income,Low,1.0


In [21]:
#Get count of characteristics in level_2
demo_data_df.groupby(["level_2"]).count()["user_id"]

level_2
Age       14068
Gender    18071
Income    70606
Name: user_id, dtype: int64

In [22]:
#Sort demographics data by user_id, in ascending order, and confidence_score than in descending order. This puts the
#the demographic characteristic that we are most confident with (higher confident score) at the top.
sorted_demo_df = demo_data_df.sort_values(['user_id', 'confidence_score'], ascending=[True, False])
sorted_demo_df.head(50)

,user_id,platform,level_1,level_2,level_3,confidence_score
2725,0,iOS,Demographics,Income,High,1.000000
22810,1,android,Demographics,Income,Medium,1.000000
6437,2,android,Demographics,Income,Medium,1.000000
95214,2,android,Demographics,Income,Low,1.000000
48246,3,android,Demographics,Income,Low,1.000000
12091,6,android,Demographics,Income,Medium,1.000000
40154,7,iOS,Demographics,Income,High,1.000000
85168,9,android,Demographics,Income,Low,1.000000
2703,11,android,Demographics,Income,Low,1.000000
85243,11,android,Demographics,Income,Medium,1.000000


In [23]:
#Create AGE dataframe.
age_df = sorted_demo_df[(sorted_demo_df['level_2'] == 'Age')]
age_df

,user_id,platform,level_1,level_2,level_3,confidence_score
16365,35,android,Demographics,Age,18 - 24,1.000000
29899,58,android,Demographics,Age,25 - 34,0.999957
2591,59,android,Demographics,Age,25 - 34,0.999999
51710,67,iOS,Demographics,Age,18 - 24,1.000000
57201,70,android,Demographics,Age,25 - 34,1.000000
...,...,...,...,...,...,...
15193,1709396984279,android,Demographics,Age,< 18,1.000000
78596,1709396984299,iOS,Demographics,Age,18 - 24,1.000000
93011,1709396984313,android,Demographics,Age,18 - 24,1.000000
90719,1709396984318,iOS,Demographics,Age,18 - 24,1.000000


In [24]:
#Make final AGE dataframe with only user_id and Age levels. Rename column and drop duplicates of user_id. 
final_age_df = age_df.drop(columns = ['platform','level_1','level_2','confidence_score']).rename(columns={"level_3":"age"}).drop_duplicates(subset=['user_id'], keep='first', inplace=False)
final_age_df

,user_id,age
16365,35,18 - 24
29899,58,25 - 34
2591,59,25 - 34
51710,67,18 - 24
57201,70,25 - 34
...,...,...
15193,1709396984279,< 18
78596,1709396984299,18 - 24
93011,1709396984313,18 - 24
90719,1709396984318,18 - 24


In [25]:
#Create GENDER dataframe.
gender_df = sorted_demo_df[(sorted_demo_df['level_2'] == 'Gender')]
gender_df

,user_id,platform,level_1,level_2,level_3,confidence_score
90047,27,web,Demographics,Gender,Female,0.980000
82227,28,android,Demographics,Gender,Female,0.820000
94261,30,android,Demographics,Gender,Female,0.990000
93628,35,android,Demographics,Gender,Male,0.835705
18400,39,android,Demographics,Gender,Male,0.990000
...,...,...,...,...,...,...
59894,1709396984279,android,Demographics,Gender,Female,0.998309
41417,1709396984299,iOS,Demographics,Gender,Female,0.999196
51959,1709396984313,android,Demographics,Gender,Male,0.772061
64710,1709396984318,iOS,Demographics,Gender,Female,0.993463


In [26]:
#Make final GENDER dataframe with only user_id and gender. Rename column. Drop any duplicates. 
final_gender_df = gender_df.drop(columns = ['platform','level_1','level_2','confidence_score']).rename(columns={"level_3":"gender"}).drop_duplicates(subset=['user_id'], keep='first', inplace=False)
final_gender_df

,user_id,gender
90047,27,Female
82227,28,Female
94261,30,Female
93628,35,Male
18400,39,Male
...,...,...
59894,1709396984279,Female
41417,1709396984299,Female
51959,1709396984313,Male
64710,1709396984318,Female


In [27]:
#Create INCOME dataframe.
income_df = sorted_demo_df[(sorted_demo_df['level_2'] == 'Income')]
income_df

,user_id,platform,level_1,level_2,level_3,confidence_score
2725,0,iOS,Demographics,Income,High,1.0
22810,1,android,Demographics,Income,Medium,1.0
6437,2,android,Demographics,Income,Medium,1.0
95214,2,android,Demographics,Income,Low,1.0
48246,3,android,Demographics,Income,Low,1.0
...,...,...,...,...,...,...
75990,1709396984327,android,Demographics,Income,Low,1.0
23051,1709396984328,android,Demographics,Income,Medium,1.0
36510,1709396984329,android,Demographics,Income,Medium,1.0
30516,1709396984331,android,Demographics,Income,Low,1.0


From the above dataframe, can see that there can be two income levels present for one user_id (e.g. user_id 2 has medium and low income levels listed). 

To see how many of these occur:

In [28]:
# Select all duplicate rows based on user_id
duplicate_users_df = income_df[income_df.duplicated(['user_id'])]
duplicate_users_df

,user_id,platform,level_1,level_2,level_3,confidence_score
95214,2,android,Demographics,Income,Low,1.0
85243,11,android,Demographics,Income,Medium,1.0
72550,71,android,Demographics,Income,Low,1.0
69083,288,android,Demographics,Income,High,1.0
70497,389,android,Demographics,Income,Low,1.0
...,...,...,...,...,...,...
101327,1709396983832,android,Demographics,Income,Medium,1.0
91647,1709396983860,android,Demographics,Income,Medium,1.0
94101,1709396983866,android,Demographics,Income,High,1.0
88845,1709396983981,iOS,Demographics,Income,Medium,1.0


In [29]:
# Drop any user_id's that have duplicate income level answers.
drop_duplicate_users_df = income_df.drop_duplicates(subset=['user_id'],keep=False)
drop_duplicate_users_df

,user_id,platform,level_1,level_2,level_3,confidence_score
2725,0,iOS,Demographics,Income,High,1.0
22810,1,android,Demographics,Income,Medium,1.0
48246,3,android,Demographics,Income,Low,1.0
12091,6,android,Demographics,Income,Medium,1.0
40154,7,iOS,Demographics,Income,High,1.0
...,...,...,...,...,...,...
75990,1709396984327,android,Demographics,Income,Low,1.0
23051,1709396984328,android,Demographics,Income,Medium,1.0
36510,1709396984329,android,Demographics,Income,Medium,1.0
30516,1709396984331,android,Demographics,Income,Low,1.0


In [30]:
#Make final INCOME dataframe with only user_id and income. Rename column. Drop any duplicates. 
final_income_df = drop_duplicate_users_df.drop(columns = ['platform','level_1','level_2','confidence_score']).rename(columns={"level_3":"income_level"})
final_income_df

,user_id,income_level
2725,0,High
22810,1,Medium
48246,3,Low
12091,6,Medium
40154,7,High
...,...,...
75990,1709396984327,Low
23051,1709396984328,Medium
36510,1709396984329,Medium
30516,1709396984331,Low


In [31]:
from functools import reduce

In [32]:
#Merge to make final demographic dataframe. Combine the age, income, and gender demographics.
df_to_merge = [final_gender_df, final_income_df, final_age_df]
final_demo_df = reduce(lambda  left,right: pd.merge(left,right,on=['user_id'],
                                            how='outer'), df_to_merge)
final_demo_df

,user_id,gender,income_level,age
0,27,Female,Low,NaN
1,28,Female,Medium,NaN
2,30,Female,High,NaN
3,35,Male,Low,18 - 24
4,39,Male,Medium,NaN
...,...,...,...,...
70864,1709396984327,NaN,Low,NaN
70865,1709396984328,NaN,Medium,NaN
70866,1709396984329,NaN,Medium,NaN
70867,1709396984331,NaN,Low,NaN


In [33]:
#To count the amount of NaN values in the dataframe.
final_demo_df.isna().sum()

user_id             0
gender          52798
income_level     3605
age             56801
dtype: int64

In [34]:
#To drop any rows containing NaN.
final_df = final_demo_df.dropna().reset_index(drop=True)

In [35]:
#Export the final_demo_df to csv
final_df.to_csv('final_demographics.csv')

In [36]:
# writing to Excel
datatoexcel2 = pd.ExcelWriter('demographics_final_final.xlsx')
  
# write DataFrame to excel
final_df.to_excel(datatoexcel2)
  
# save the excel
datatoexcel2.save()

In [37]:
#See preference characteristics:
preferences_df

,user_id,platform,level_1,level_2,level_3,confidence_score
459980,0,iOS,Psychographics,iflix Viewing Behaviour,casual,1.00
300772,1,android,Psychographics,iflix Viewing Behaviour,casual,1.00
85066,2,android,Psychographics,iflix Viewing Behaviour,addict,1.00
364315,3,android,Psychographics,iflix Viewing Behaviour,casual,1.00
127666,4,android-tv,Psychographics,Movies Lovers,Comedy Movies Fans,0.76
...,...,...,...,...,...,...
252006,1709396984329,android,Psychographics,Mobile Enthusiasts,High Data Users,0.53
416896,1709396984330,android,Psychographics,Mobile Enthusiasts,High Data Users,0.08
340584,1709396984331,android,Psychographics,Mobile Enthusiasts,High Data Users,0.68
314367,1709396984332,android,Psychographics,Mobile Enthusiasts,High Data Users,0.03


In [38]:
#Clean up preferences. Remove iflix Viewing Behaviour rows to its own dataframe. And the 
#user_ids will be defined by level_2 values and level_3 values for those with "iflix viewing
#behaviour" in level_2.

fans_df = preferences_df[preferences_df.level_2 != 'iflix Viewing Behaviour'].drop(columns = ['level_1','level_3','confidence_score']).rename(columns={"level_2":"characteristics"})
fans_df

,user_id,platform,characteristics
127666,4,android-tv,Movies Lovers
4492,9,android,TV Lovers
10763,11,android,Travellers
187261,12,android,Travellers
83856,15,android,Movies Lovers
...,...,...,...
252006,1709396984329,android,Mobile Enthusiasts
416896,1709396984330,android,Mobile Enthusiasts
340584,1709396984331,android,Mobile Enthusiasts
314367,1709396984332,android,Mobile Enthusiasts


In [39]:
#Create new dataframe for casual or addict behaviour and include that as a characteristic.

behaviour_df = preferences_df.loc[preferences_df['level_2'] == 'iflix Viewing Behaviour']
behaviour_df

,user_id,platform,level_1,level_2,level_3,confidence_score
459980,0,iOS,Psychographics,iflix Viewing Behaviour,casual,1.0
300772,1,android,Psychographics,iflix Viewing Behaviour,casual,1.0
85066,2,android,Psychographics,iflix Viewing Behaviour,addict,1.0
364315,3,android,Psychographics,iflix Viewing Behaviour,casual,1.0
320261,5,android-tv,Psychographics,iflix Viewing Behaviour,casual,1.0
...,...,...,...,...,...,...
309870,1709396984300,android,Psychographics,iflix Viewing Behaviour,casual,1.0
30802,1709396984308,android,Psychographics,iflix Viewing Behaviour,casual,1.0
438417,1709396984311,android,Psychographics,iflix Viewing Behaviour,casual,1.0
16729,1709396984312,android,Psychographics,iflix Viewing Behaviour,casual,1.0


In [40]:
#Clean up behaviour_df.


behaviour_df = behaviour_df.drop(columns = ['level_1','level_2','confidence_score']).rename(columns={"level_3":"characteristics"})

In [41]:
behaviour_df

,user_id,platform,characteristics
459980,0,iOS,casual
300772,1,android,casual
85066,2,android,addict
364315,3,android,casual
320261,5,android-tv,casual
...,...,...,...
309870,1709396984300,android,casual
30802,1709396984308,android,casual
438417,1709396984311,android,casual
16729,1709396984312,android,casual


In [42]:
#Add behaviour_df to fans_df to get the characteristics of all users.
characteristics_df = behaviour_df.append(fans_df)
characteristics_df

,user_id,platform,characteristics
459980,0,iOS,casual
300772,1,android,casual
85066,2,android,addict
364315,3,android,casual
320261,5,android-tv,casual
...,...,...,...
252006,1709396984329,android,Mobile Enthusiasts
416896,1709396984330,android,Mobile Enthusiasts
340584,1709396984331,android,Mobile Enthusiasts
314367,1709396984332,android,Mobile Enthusiasts


In [44]:
characteristics_df = characteristics_df.sort_values(by=['user_id'])
characteristics_df.head(40)

,user_id,platform,characteristics
459980,0,iOS,casual
300772,1,android,casual
85066,2,android,addict
364315,3,android,casual
127666,4,android-tv,Movies Lovers
320261,5,android-tv,casual
188550,6,android,player
75248,7,iOS,casual
185056,8,Panasonic,casual
4492,9,android,TV Lovers


In [45]:
#Export the characteristics_df to csv
characteristics_df.to_csv('characteristics.csv')

In [46]:
# writing to Excel
datatoexcel3 = pd.ExcelWriter('characteristics.xlsx')
  
# write DataFrame to excel
characteristics_df.to_excel(datatoexcel3)
  
# save the excel
datatoexcel3.save()